# Regime-Conditioned Model Training
**Purpose**: Test whether training separate models per market regime outperforms a single global model

## Hypothesis
Standard features may have no predictive power globally but could be useful within specific
market regimes (bull/bear/sideways). Regime-conditioned models train on regime-specific subsets
of the data and are selected at inference time based on the detected regime.

## Pipeline
1. Load data & detect regimes using HMM
2. Analyze regime distribution and characteristics
3. Train per-regime models (XGBoost)
4. Build regime-switching ensemble
5. Compare: global model vs regime-conditioned vs naive baseline

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import json
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import joblib

# Add project root for HMM detector
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))
from src.regime.hmm_detector import HMMRegimeDetector

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Libraries loaded')

In [ ]:
# === Configuration ===
DATA_DIR = Path('/home/archy/Desktop/Server/FinancialData/model_regime_comparison/data/prepared')
MODEL_DIR = Path('/home/archy/Desktop/Server/FinancialData/model_regime_comparison/models')
MODEL_DIR.mkdir(parents=True, exist_ok=True)

TARGET = 'forward_return_30d'
EXCLUDE_COLS = ['ticker', 'date', 'close', 'forward_return_5d', 'forward_return_30d']
RANDOM_STATE = 42

# Minimum samples per regime to train a dedicated model
MIN_REGIME_SAMPLES = 100

print(f'Target: {TARGET}')

## 1. Load Data

In [ ]:
train_df = pd.read_parquet(DATA_DIR / 'finance_train.parquet')
val_df = pd.read_parquet(DATA_DIR / 'finance_val.parquet')
test_df = pd.read_parquet(DATA_DIR / 'finance_test.parquet')

# Load selected features
features_path = MODEL_DIR / 'selected_features.json'
if features_path.exists():
    with open(features_path) as f:
        feat_info = json.load(f)
    selected_features = feat_info['selected_features']
    print(f'Loaded {len(selected_features)} selected features from 03a')
else:
    all_feature_cols = [c for c in train_df.columns if c not in EXCLUDE_COLS]
    selected_features = train_df[all_feature_cols].select_dtypes(include=[np.number]).columns.tolist()
    print(f'Using all {len(selected_features)} numeric features')

print(f'Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}')

## 2. Regime Detection
Train HMM on per-ticker close prices to assign bull/bear/sideways labels to each row.

In [ ]:
from hmmlearn import hmm as hmmlearn_hmm

def _fit_hmm_safe(prices, n_states=3, window=20, random_state=42):
    """Fit HMM with diagonal covariance (more stable) and return regime history."""
    returns = prices.pct_change().dropna()
    volatility = returns.rolling(window=window).std().dropna()
    aligned_returns = returns[volatility.index]
    features = np.column_stack([aligned_returns.values, volatility.values])
    dates = prices.index[window:]

    model = hmmlearn_hmm.GaussianHMM(
        n_components=n_states, covariance_type='diag',
        n_iter=100, random_state=random_state)
    model.fit(features)

    states = model.predict(features)
    probs = model.predict_proba(features)

    # Label states by mean return
    state_means = [features[states == s, 0].mean() for s in range(n_states)]
    sorted_idx = np.argsort(state_means)
    labels = {sorted_idx[0]: 'bear', sorted_idx[1]: 'sideways', sorted_idx[2]: 'bull'}

    records = []
    for i, (dt, si) in enumerate(zip(dates, states)):
        records.append({'date': dt, 'regime': labels[si], 'confidence': probs[i, si]})
    return pd.DataFrame(records).set_index('date')

def assign_regimes(df, window=20):
    """
    Assign regime labels to each row by fitting an HMM per ticker on close prices.
    Returns df with added 'regime' and 'regime_confidence' columns.
    """
    regime_records = []

    for ticker in df['ticker'].unique():
        ticker_df = df[df['ticker'] == ticker].sort_values('date')
        prices = ticker_df.set_index('date')['close']

        if len(prices) < window + 50:
            print(f'  {ticker}: skipping (only {len(prices)} rows)')
            continue

        try:
            history = _fit_hmm_safe(prices, n_states=3, window=window, random_state=RANDOM_STATE)
        except Exception as e:
            print(f'  {ticker}: HMM failed ({e}), skipping')
            continue

        for date_idx, row in history.iterrows():
            regime_records.append({
                'ticker': ticker,
                'date': date_idx,
                'regime': row['regime'],
                'regime_confidence': row['confidence']
            })

    regime_df = pd.DataFrame(regime_records)
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    regime_df['date'] = pd.to_datetime(regime_df['date'])
    merged = df.merge(regime_df, on=['ticker', 'date'], how='left')
    return merged

print('Assigning regimes to training set...')
train_df = assign_regimes(train_df)
print(f'  Rows with regime: {train_df["regime"].notna().sum()} / {len(train_df)}')

print('\nAssigning regimes to validation set...')
val_df = assign_regimes(val_df)
print(f'  Rows with regime: {val_df["regime"].notna().sum()} / {len(val_df)}')

print('\nAssigning regimes to test set...')
test_df = assign_regimes(test_df)
print(f'  Rows with regime: {test_df["regime"].notna().sum()} / {len(test_df)}')

In [ ]:
# Drop rows without regime assignment
train_df = train_df.dropna(subset=['regime'])
val_df = val_df.dropna(subset=['regime'])
test_df = test_df.dropna(subset=['regime'])

print('After dropping rows without regime:')
print(f'  Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}')

## 3. Regime Distribution Analysis

In [ ]:
# Regime distribution across splits
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
regime_order = ['bull', 'sideways', 'bear']
colors = {'bull': 'green', 'sideways': 'gray', 'bear': 'red'}

for ax, (name, df) in zip(axes, [('Train', train_df), ('Validation', val_df), ('Test', test_df)]):
    counts = df['regime'].value_counts()
    bars = ax.bar([r for r in regime_order if r in counts.index],
                  [counts.get(r, 0) for r in regime_order if r in counts.index],
                  color=[colors[r] for r in regime_order if r in counts.index], alpha=0.7)
    ax.set_title(f'{name} ({len(df)} rows)')
    ax.set_ylabel('Count')
    for bar, r in zip(bars, [r for r in regime_order if r in counts.index]):
        pct = counts.get(r, 0) / len(df) * 100
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                f'{pct:.1f}%', ha='center', va='bottom', fontsize=11)
    ax.grid(True, alpha=0.3)

plt.suptitle('Regime Distribution Across Data Splits', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Target distribution per regime
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for ax, regime in zip(axes, regime_order):
    regime_data = train_df[train_df['regime'] == regime][TARGET].dropna()
    if len(regime_data) > 0:
        ax.hist(regime_data, bins=40, edgecolor='black', alpha=0.7, color=colors[regime])
        ax.axvline(regime_data.mean(), color='black', linestyle='--', lw=2,
                   label=f'mean={regime_data.mean():.4f}')
        ax.axvline(0, color='gray', linestyle=':', lw=1)
        ax.set_title(f'{regime.upper()} (n={len(regime_data)}, σ={regime_data.std():.4f})')
        ax.set_xlabel(TARGET)
        ax.legend()
        ax.grid(True, alpha=0.3)

plt.suptitle(f'Target Distribution by Regime (Train)', fontsize=14)
plt.tight_layout()
plt.show()

# Print summary stats
print(f'{"Regime":<12s} {"Count":>6s} {"Mean":>10s} {"Std":>10s} {"Median":>10s}')
print('-' * 50)
for regime in regime_order:
    rd = train_df[train_df['regime'] == regime][TARGET].dropna()
    if len(rd) > 0:
        print(f'{regime:<12s} {len(rd):>6d} {rd.mean():>10.4f} {rd.std():>10.4f} {rd.median():>10.4f}')

## 4. Prepare Data

In [ ]:
def prepare_Xy(df, features, target):
    """Extract X, y from df, fill NaN, drop target NaN rows."""
    X = df[features].copy()
    y = df[target].copy()
    train_medians = X.median()
    X = X.fillna(train_medians)
    # Drop remaining NaN columns
    still_nan = X.columns[X.isnull().any()].tolist()
    if still_nan:
        X = X.drop(columns=still_nan)
    mask = ~y.isna()
    return X[mask], y[mask], df[mask]

X_train, y_train, train_meta = prepare_Xy(train_df, selected_features, TARGET)
X_val, y_val, val_meta = prepare_Xy(val_df, selected_features, TARGET)
X_test, y_test, test_meta = prepare_Xy(test_df, selected_features, TARGET)

# Use consistent feature set
common_features = list(X_train.columns)
X_val = X_val[common_features]
X_test = X_test[common_features]

print(f'Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}')

## 5. Train Global Baseline Model

In [ ]:
# Best XGBoost config from notebook 04 (low learning rate, moderate depth)
global_model = xgb.XGBRegressor(
    n_estimators=500, max_depth=3, learning_rate=0.01,
    subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1, reg_lambda=1.0,
    random_state=RANDOM_STATE, n_jobs=-1, verbosity=0
)
global_model.fit(X_train, y_train)

global_val_pred = global_model.predict(X_val)
global_test_pred = global_model.predict(X_test)

print('Global XGBoost Baseline:')
print(f'  Val  RMSE={np.sqrt(mean_squared_error(y_val, global_val_pred)):.6f}  R²={r2_score(y_val, global_val_pred):.6f}')
print(f'  Test RMSE={np.sqrt(mean_squared_error(y_test, global_test_pred)):.6f}  R²={r2_score(y_test, global_test_pred):.6f}')

## 6. Train Per-Regime Models

In [ ]:
regime_models = {}
regime_results = []

for regime in regime_order:
    # Get regime subsets
    train_mask = train_meta['regime'].values == regime
    val_mask = val_meta['regime'].values == regime

    X_tr_r = X_train[train_mask]
    y_tr_r = y_train[train_mask]
    X_v_r = X_val[val_mask]
    y_v_r = y_val[val_mask]

    print(f'\n{regime.upper()} regime: train={len(X_tr_r)}, val={len(X_v_r)}')

    if len(X_tr_r) < MIN_REGIME_SAMPLES:
        print(f'  ⚠ Too few training samples, using global model as fallback')
        regime_models[regime] = global_model
        continue

    # Train regime-specific model
    model = xgb.XGBRegressor(
        n_estimators=500, max_depth=3, learning_rate=0.01,
        subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1, reg_lambda=1.0,
        random_state=RANDOM_STATE, n_jobs=-1, verbosity=0
    )
    model.fit(X_tr_r, y_tr_r)
    regime_models[regime] = model

    # Evaluate on regime subset
    y_tr_pred = model.predict(X_tr_r)
    tr_rmse = np.sqrt(mean_squared_error(y_tr_r, y_tr_pred))
    tr_r2 = r2_score(y_tr_r, y_tr_pred)

    if len(X_v_r) > 0:
        y_v_pred = model.predict(X_v_r)
        v_rmse = np.sqrt(mean_squared_error(y_v_r, y_v_pred))
        v_r2 = r2_score(y_v_r, y_v_pred)
        print(f'  Train RMSE={tr_rmse:.6f}  R²={tr_r2:.6f}')
        print(f'  Val   RMSE={v_rmse:.6f}  R²={v_r2:.6f}')
    else:
        v_rmse, v_r2 = np.nan, np.nan
        print(f'  Train RMSE={tr_rmse:.6f}  R²={tr_r2:.6f}')
        print(f'  No validation samples for this regime')

    regime_results.append({
        'Regime': regime, 'Train Samples': len(X_tr_r), 'Val Samples': len(X_v_r),
        'Train RMSE': tr_rmse, 'Train R²': tr_r2,
        'Val RMSE': v_rmse, 'Val R²': v_r2
    })

print('\n✓ Per-regime models trained')

## 7. Regime-Switching Predictions

In [ ]:
def regime_switching_predict(X, regime_labels, regime_models, fallback_model):
    """
    Generate predictions by selecting the appropriate model for each row's regime.
    """
    predictions = np.zeros(len(X))
    for regime, model in regime_models.items():
        mask = regime_labels == regime
        if mask.any():
            predictions[mask] = model.predict(X[mask])
    # Fallback for any unassigned rows
    unassigned = ~np.isin(regime_labels, list(regime_models.keys()))
    if unassigned.any():
        predictions[unassigned] = fallback_model.predict(X[unassigned])
    return predictions

# Validation predictions
val_regimes = val_meta['regime'].values
regime_val_pred = regime_switching_predict(X_val, val_regimes, regime_models, global_model)

# Test predictions
test_regimes = test_meta['regime'].values
regime_test_pred = regime_switching_predict(X_test, test_regimes, regime_models, global_model)

print('Regime-Switching Ensemble:')
print(f'  Val  RMSE={np.sqrt(mean_squared_error(y_val, regime_val_pred)):.6f}  R²={r2_score(y_val, regime_val_pred):.6f}')
print(f'  Test RMSE={np.sqrt(mean_squared_error(y_test, regime_test_pred)):.6f}  R²={r2_score(y_test, regime_test_pred):.6f}')

## 8. Full Comparison

In [ ]:
# Naive baseline: predict mean of training target
naive_pred = np.full(len(y_test), y_train.mean())

# Per-regime naive: predict mean of training target within each regime
regime_naive_pred = np.zeros(len(y_test))
for regime in regime_order:
    train_regime_mask = train_meta['regime'].values == regime
    test_regime_mask = test_meta['regime'].values == regime
    if train_regime_mask.any() and test_regime_mask.any():
        regime_mean = y_train[train_regime_mask].mean()
        regime_naive_pred[test_regime_mask] = regime_mean
# Fallback for unmatched
unmatched = regime_naive_pred == 0
if unmatched.any():
    regime_naive_pred[unmatched] = y_train.mean()

# Compile all test results
comparison = {
    'Naive Mean': naive_pred,
    'Regime-Naive Mean': regime_naive_pred,
    'Global XGBoost': global_test_pred,
    'Regime-Conditioned XGBoost': regime_test_pred,
}

print(f'{"Model":<32s} {"Test RMSE":>10s} {"Test MAE":>10s} {"Test R²":>10s}')
print('-' * 65)
for name, preds in comparison.items():
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    print(f'{name:<32s} {rmse:>10.6f} {mae:>10.6f} {r2:>10.6f}')

In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

model_names = list(comparison.keys())
rmses = [np.sqrt(mean_squared_error(y_test, p)) for p in comparison.values()]
r2s = [r2_score(y_test, p) for p in comparison.values()]

# RMSE
bar_colors = ['lightgray', 'lightblue', 'steelblue', 'darkorange']
axes[0].barh(model_names, rmses, color=bar_colors, alpha=0.8)
axes[0].set_xlabel('Test RMSE (lower is better)')
axes[0].set_title('RMSE Comparison')
axes[0].grid(True, alpha=0.3, axis='x')

# R²
axes[1].barh(model_names, r2s, color=bar_colors, alpha=0.8)
axes[1].axvline(0, color='red', linestyle='--', lw=1, label='R²=0 (mean baseline)')
axes[1].set_xlabel('Test R² (higher is better)')
axes[1].set_title('R² Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='x')

plt.suptitle('Global vs Regime-Conditioned Models', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Per-regime breakdown on test set
print(f'{"Regime":<10s} {"N":>5s}  {"Global RMSE":>12s} {"Global R²":>10s}  {"Regime RMSE":>12s} {"Regime R²":>10s}  {"Δ R²":>8s}')
print('-' * 80)

for regime in regime_order:
    mask = test_meta['regime'].values == regime
    if not mask.any():
        continue
    y_r = y_test[mask]
    g_pred = global_test_pred[mask]
    r_pred = regime_test_pred[mask]

    g_rmse = np.sqrt(mean_squared_error(y_r, g_pred))
    g_r2 = r2_score(y_r, g_pred)
    r_rmse = np.sqrt(mean_squared_error(y_r, r_pred))
    r_r2 = r2_score(y_r, r_pred)
    delta = r_r2 - g_r2

    print(f'{regime:<10s} {mask.sum():>5d}  {g_rmse:>12.6f} {g_r2:>10.6f}  {r_rmse:>12.6f} {r_r2:>10.6f}  {delta:>+8.6f}')

## 9. Feature Importance per Regime

In [ ]:
regime_with_models = [r for r in regime_order if r in regime_models and regime_models[r] is not global_model]

if len(regime_with_models) > 0:
    fig, axes = plt.subplots(1, len(regime_with_models),
                             figsize=(7 * len(regime_with_models), max(6, len(common_features) * 0.35)))
    if len(regime_with_models) == 1:
        axes = [axes]

    for ax, regime in zip(axes, regime_with_models):
        model = regime_models[regime]
        imp = model.feature_importances_
        imp_df = pd.DataFrame({'Feature': common_features, 'Importance': imp}).sort_values('Importance', ascending=True)
        ax.barh(range(len(imp_df)), imp_df['Importance'], color=colors[regime], alpha=0.7)
        ax.set_yticks(range(len(imp_df)))
        ax.set_yticklabels(imp_df['Feature'])
        ax.set_xlabel('Importance')
        ax.set_title(f'{regime.upper()} Regime')
        ax.grid(True, alpha=0.3, axis='x')

    plt.suptitle('Feature Importance per Regime', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print('No dedicated regime models were trained (all fell back to global model).')

## 10. Save Artifacts

In [ ]:
# Save regime models
for regime, model in regime_models.items():
    model_path = MODEL_DIR / f'regime_model_{regime}.pkl'
    joblib.dump(model, model_path)
    print(f'✓ Saved {regime} model to {model_path}')

# Save comparison metadata
regime_meta = {
    'target': TARGET,
    'features': common_features,
    'regimes': regime_order,
    'comparison': {}
}
for name, preds in comparison.items():
    regime_meta['comparison'][name] = {
        'test_rmse': float(np.sqrt(mean_squared_error(y_test, preds))),
        'test_mae': float(mean_absolute_error(y_test, preds)),
        'test_r2': float(r2_score(y_test, preds)),
    }

meta_path = MODEL_DIR / 'regime_comparison_metadata.json'
with open(meta_path, 'w') as f:
    json.dump(regime_meta, f, indent=2)
print(f'\n✓ Comparison metadata saved to {meta_path}')

print(f'\n{"="*60}')
print('Regime Comparison Summary')
print(f'{"="*60}')
for name, metrics in regime_meta['comparison'].items():
    print(f'  {name:<32s}  R²={metrics["test_r2"]:+.6f}')
print(f'{"="*60}')